In [ ]:
from typing import Literal # Requires Python 3.8+
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

from visualization import visualize_brain_surface
from smoothing import smooth_surface_graph
from watershed import watershed_by_flooding
from gradient import build_mesh_graph

from group_analysis import create_random_parcels, homogeneity_craddock_rt


In [ ]:
def extract_fmri_timeseries(hemisphere: Literal["lh", "rh"],
                            run = Literal["1","2"]):
    # Load all of the surf fmri data
    surf_fmri_list = []
    dataset_dir = r"D:\Data_Conn_Preproc\PPSFACE_N18"
    for i in range(1,19):
        # if i == 5: # Subject 5 is missing
        #     continue
        subject = f"{i:02d}"
        subj_dir = dataset_dir + r"\sub-" + subject
        fmri_path = subj_dir + f"\\func\surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
        surf_fmri_img = nib.load(fmri_path)
        surf_fmri = surf_fmri_img.get_fdata()
        surf_fmri = np.squeeze(surf_fmri) # just rearrange the MGH data
        # Normilize the data
        surf_fmri = (surf_fmri - np.mean(surf_fmri, axis=1, keepdims=True)) / np.std(surf_fmri, axis=1, keepdims=True)
        # Replace nan values with 0
        surf_fmri = np.nan_to_num(surf_fmri)
        surf_fmri_list.append(surf_fmri)
    return surf_fmri_list

def extract_parcellation(hemi: Literal["lh", "rh"], run = Literal["1","2"]):
    # Load the parcellation
    bound_path = f"D:\Data_Conn_Preproc\PPSFACE_N20\\boundary_smoothed_map_allsub_run{run}_{hemi}.npy"
    grad_path = f"D:\Data_Conn_Preproc\PPSFACE_N20\gradients_smoothed_allsub_run{run}_{hemi}.npy"
    # bound_path = f"D:\Data_Conn_Preproc\PPSFACE_N18\group_parcellation\\boundary_map_group_run{run}_{hemi}.npy"
    # grad_path = f"D:\Data_Conn_Preproc\PPSFACE_N18\group_parcellation\gradients_smoothed_group_run{run}_{hemi}.npy"
    bound = np.load(bound_path)
    grad = np.load(grad_path)
    return bound, grad

## Show both hemisphere boundary maps

In [ ]:
from visualization import combine_surfaces
# Load the parcellation for lh run1
run = "1"

inf_lh_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\lh.white"
inf_rh_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\\rh.white"
# surface_path = r"D:\Data_Conn_Preproc\fsaverage6\surf\rh.inflated"
coords_lh, faces_lh = nib.freesurfer.read_geometry(inf_lh_path)
coords_rh, faces_rh = nib.freesurfer.read_geometry(inf_rh_path)

# Extract bounardy and grad maps
bound_lh, _ = extract_parcellation('lh', run)
bound_rh, _ = extract_parcellation('rh', run)

# Combine surface meshes
coords_c, faces_c, scalar_c = combine_surfaces(coords_lh, faces_lh, bound_lh,
                                              coords_rh, faces_rh, bound_rh)

visualize_brain_surface(coords_c, faces_c, scalar_c, title='Boundary map ', cmap='cold_hot')



## Dice Coef comparison

In [ ]:
# Dice coeff run 1 vs run 2 for the boundary maps
from group_analysis import dice_coefficient
hemi = "lh"
run = "1"

surface_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.white"
inf_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.inflated"
# surface_path = r"D:\Data_Conn_Preproc\fsaverage6\surf\rh.inflated"
coords, faces = nib.freesurfer.read_geometry(surface_path)
coords_, faces_ = nib.freesurfer.read_geometry(inf_path)
graph = build_mesh_graph(faces)


bound, grad = extract_parcellation(hemi, run)
bound_smoothed = smooth_surface_graph(graph, bound, iterations=10)
label_bound_run1 = watershed_by_flooding(graph, bound_smoothed)


run = "2"
bound, grad = extract_parcellation(hemi, run)
bound_smoothed = smooth_surface_graph(graph, bound, iterations=10)
label_bound_run2 = watershed_by_flooding(graph, bound_smoothed)

dice_p = dice_coefficient((label_bound_run1>=0)*1, (label_bound_run2>=0)*1)
dice_b = dice_coefficient((label_bound_run1<0)*1, (label_bound_run2<0)*1)

print(f"Number of parcels run1: {(np.unique(label_bound_run1)>=0).sum()}")
print(f"Number of parcels run2: {(np.unique(label_bound_run2)>=0).sum()}")


print(f"Run 1-2 Dice coefficient on parcels: {dice_p}")
print(f"Run 1-2 Dice coefficient on boundaries: {dice_b}")


In [ ]:
dice_rnd = []
dice_rnd_parc = []
for i in range(100):
    # Compare the Dice indice for random parcels of same size
    rnd_parc_1 = create_random_parcels(graph,n_clusters=(np.unique(label_bound_run1)>=0).sum())
    rnd_parc_2 = create_random_parcels(graph,n_clusters=(np.unique(label_bound_run2)>=0).sum())

    dice_p_ = dice_coefficient((rnd_parc_1>=0)*1, (rnd_parc_2>=0)*1)
    # dice_b_ = dice_coefficient((rnd_parc_1<0)*1, (rnd_parc_2<0)*1)
    dice_rnd_parc.append(dice_p_)
    # dice_rnd.append(dice_b_)
    
    # print(f"Random 1-2 Dice coefficient on parcels: {dice_b_}")
    print(f"Random 1-2 Dice coefficient on parcels: {dice_p_}")

In [ ]:
# plot the homogenity scores
x_null = np.ones(len(dice_rnd_parc)) + np.random.uniform(-0.05, 0.05, len(dice_rnd_parc))
x_model = np.array([1]) 

plt.figure(figsize=(2, 6))
plt.scatter(x_model, [dice_p], color='red', marker='D', s=100, label='Proposed Model')
plt.scatter(x_null, dice_rnd_parc, color='blue', alpha=0.7, label='Null Model')

# Customizing the plot
plt.xticks([1],'')  # Single x-tick since both are on the same column
plt.ylabel('Dice Coefficient')
plt.title('Consistency Between Run 1 and Run 2, LH')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Show the plot
plt.show()

## Create an homogenity metrix like craddock

In [ ]:
# Load the parcellation for lh run1
hemi = "lh"
run = "1"
surface_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.white"
inf_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.inflated"
# surface_path = r"D:\Data_Conn_Preproc\fsaverage6\surf\rh.inflated"
coords, faces = nib.freesurfer.read_geometry(surface_path)
coords_, faces_ = nib.freesurfer.read_geometry(inf_path)
graph = build_mesh_graph(faces)

# Extract bounardy and grad maps
bound, grad = extract_parcellation(hemi, run)

bound_smoothed = smooth_surface_graph(graph, bound, iterations=10)
grad_smoothed = smooth_surface_graph(graph, np.mean(grad,axis=1), iterations=10)

label_grad = watershed_by_flooding(graph, grad_smoothed)
label_bound = watershed_by_flooding(graph, bound_smoothed)

# Load all of the surf fmri data form run 1 lh
surf_fmri_list = extract_fmri_timeseries(hemi, run)

In [ ]:
print('mean and var', surf_fmri_list[2].mean(), surf_fmri_list[2].var())
print('list length', len(surf_fmri_list))

In [ ]:
visualize_brain_surface(coords_, faces_, bound)

In [ ]:
homogeneity_model = homogeneity_craddock_rt(label_bound, surf_fmri_list)
print(f"Model homogeneity: {homogeneity_model}")

In [ ]:
np.unique(label_bound)

In [ ]:
# Test the null model on the
homogenity_null_list = []
for n_sample in range(1, 100):
    label_null = create_random_parcels(graph, n_clusters=(np.unique(label_bound)>0).sum()) # Create a null model
    homogeneity_null = homogeneity_craddock_rt(label_null, surf_fmri_list)
    homogenity_null_list.append(homogeneity_null)
    print(f"Null homogeneity: {homogeneity_null}")


In [ ]:
# plot the homogenity scores
x_null = np.ones(len(homogenity_null_list)) + np.random.uniform(-0.05, 0.05, len(homogenity_null_list))
x_model = np.array([1]) 

plt.figure(figsize=(2, 6))
plt.scatter(x_model, [homogeneity_model], color='red', marker='D', s=100, label='Proposed Model')
plt.scatter(x_null, homogenity_null_list, color='blue', alpha=0.7, label='Null Model')

# Customizing the plot
plt.xticks([1],'')  # Single x-tick since both are on the same column
plt.ylabel('Homogeneity Score (Craddock)')
plt.title('Distribution of Null Scores vs. Single-Run Model Score')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Show the plot
plt.show()